<a href="https://colab.research.google.com/github/Siva7/GoogleColabProject/blob/main/Kaggle/ImageClassification/IntelImageClassification/CustomNeuralNetDev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
import tensorflow as tf
import tensorflow.keras as tfk
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!mkdir /root/.kaggle/
!cp "/content/drive/My Drive/Colab Notebooks/KaggleAPI/kaggle.json" /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [5]:
!kaggle datasets download -d puneet6060/intel-image-classification

intel-image-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
import zipfile
with zipfile.ZipFile(r'/content/intel-image-classification.zip') as zfile:
  zfile.extractall(r'/content/intelImages')

#@title Default title text
#@ Image Size
All images are of size 150 * 150 

1) Use the raw data as is and analyse the predetions
2) Use data agumentation and see the difference.

In [6]:
import PIL
from PIL import Image
import numpy as np

image_sample = Image.open(r'/content/intelImages/seg_train/seg_train/buildings/0.jpg')
print(image_sample.mode)
print(image_sample.format)
print(image_sample.size)
img_arry  = np.array(image_sample)
print(img_arry.shape)
print(np.max(img_arry))

RGB
JPEG
(150, 150)
(150, 150, 3)
255


In [7]:
train_image_gen = ImageDataGenerator(rescale=1./255,
                                     rotation_range=0.4,width_shift_range=0.3,
                                     height_shift_range=0.3,
                                     shear_range=0.3)

image_fimage_flow_genlow_gen = train_image_gen.flow_from_directory(r'/content/intelImages/seg_train/seg_train',
                                                     class_mode='sparse',
                                                     batch_size=32,
                                                     target_size=(150,150))

Found 14034 images belonging to 6 classes.


In [8]:
next_batch = image_fimage_flow_genlow_gen.next()
print(type(next_batch))
print(next_batch[1].shape)
print(type(next_batch[1][0]))
print(next_batch[1][0])

<class 'tuple'>
(32,)
<class 'numpy.float32'>
5.0


In [9]:

vgg_layer = tf.keras.applications.VGG16(include_top=False, weights='imagenet',
                                        )
avg_pool_layer = tfk.layers.GlobalAveragePooling2D()(vgg_layer.output)
first_dense_layer = tfk.layers.Dense(256,activation='relu')(avg_pool_layer)
second_dense_layer = tfk.layers.Dense(128,activation='relu')(first_dense_layer)
third_dense_layer = tfk.layers.Dense(64,activation='relu')(second_dense_layer)
final_dense_layer = tfk.layers.Dense(6)(third_dense_layer)

model = tfk.models.Model(inputs=[vgg_layer.input],outputs=[final_dense_layer])
for layers in vgg_layer.layers:
  layers.trainable=False
model.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)  

In [10]:
checkpoint_filepath = r'/content/drive/My Drive/SavedWeights/ImageClassification/IntelImageClassification/model_checkpoint/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='acc',
    mode='max',
    save_best_only=True)

In [11]:
model.compile(loss=tfk.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tfk.optimizers.Adam(0.01),metrics=['acc'])

In [12]:
model.fit(image_fimage_flow_genlow_gen,epochs=10,steps_per_epoch=437,
          callbacks=[model_checkpoint_callback])

Epoch 1/10
437/437 [==============================] - 91s 207ms/step - loss: 0.7048 - acc: 0.7282
Epoch 2/10
437/437 [==============================] - 91s 208ms/step - loss: 0.5629 - acc: 0.7925
Epoch 3/10
437/437 [==============================] - 89s 204ms/step - loss: 0.5400 - acc: 0.8039
Epoch 4/10
437/437 [==============================] - 88s 202ms/step - loss: 0.5205 - acc: 0.8128
Epoch 5/10
437/437 [==============================] - 88s 202ms/step - loss: 0.5055 - acc: 0.8146
Epoch 6/10
437/437 [==============================] - 89s 204ms/step - loss: 0.4936 - acc: 0.8240
Epoch 7/10
437/437 [==============================] - 87s 200ms/step - loss: 0.4836 - acc: 0.8234
Epoch 8/10
437/437 [==============================] - 87s 199ms/step - loss: 0.4745 - acc: 0.8243
Epoch 9/10
437/437 [==============================] - 88s 201ms/step - loss: 0.4706 - acc: 0.8285
Epoch 10/10
437/437 [==============================] - 87s 200ms/step - loss: 0.4671 - acc: 0.8303


In [13]:
model.fit(image_fimage_flow_genlow_gen,epochs=10,steps_per_epoch=437,
          callbacks=[model_checkpoint_callback])

Epoch 1/10
437/437 [==============================] - 91s 208ms/step - loss: 0.4560 - acc: 0.8341
Epoch 2/10
437/437 [==============================] - 91s 208ms/step - loss: 0.4529 - acc: 0.8352
Epoch 3/10
437/437 [==============================] - 90s 207ms/step - loss: 0.4574 - acc: 0.8341
Epoch 4/10
437/437 [==============================] - 91s 207ms/step - loss: 0.4453 - acc: 0.8408
Epoch 5/10
437/437 [==============================] - 91s 207ms/step - loss: 0.4428 - acc: 0.8390
Epoch 6/10
437/437 [==============================] - 90s 205ms/step - loss: 0.4339 - acc: 0.8405
Epoch 7/10
437/437 [==============================] - 90s 206ms/step - loss: 0.4427 - acc: 0.8391
Epoch 8/10
437/437 [==============================] - 89s 204ms/step - loss: 0.4456 - acc: 0.8351
Epoch 9/10
437/437 [==============================] - 89s 203ms/step - loss: 0.4406 - acc: 0.8393
Epoch 10/10
437/437 [==============================] - 87s 199ms/step - loss: 0.4224 - acc: 0.8448


In [14]:
model.save_weights(r'/content/drive/My Drive/SavedWeights/ImageClassification/IntelImageClassification/model_weights/')

**Tetst the model on test data to see the accuracy**
1.   Generate another imageDataGenerator for Image Data
2.   Predict the output and evaluate the results



In [15]:
test_Data_Generator = ImageDataGenerator(rescale=1./255)
test_data_flow = test_Data_Generator.flow_from_directory(r'/content/intelImages/seg_test/seg_test',
                                                         target_size=(150,150),
                                                         class_mode='sparse',
                                                         batch_size=32,
                                                         )

Found 3000 images belonging to 6 classes.


In [16]:
print(test_data_flow.class_indices)

{'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}


In [17]:
model.evaluate(test_data_flow,batch_size=32,return_dict=True)

94/94 [==============================] - 7s 79ms/step - loss: 0.4260 - acc: 0.8363


{'acc': 0.8363333344459534, 'loss': 0.4259970784187317}

In [19]:
vgg_layer_two = tf.keras.applications.VGG16(include_top=False, weights='imagenet',
                                        )
avg_pool_layer_two = tfk.layers.GlobalAveragePooling2D()(vgg_layer_two.output)
first_dense_layer_two = tfk.layers.Dense(256,activation='relu')(avg_pool_layer_two)
second_dense_layer_two = tfk.layers.Dense(128,activation='relu')(first_dense_layer_two)
third_dense_layer_two = tfk.layers.Dense(64,activation='relu')(second_dense_layer_two)
final_dense_layer_two = tfk.layers.Dense(6)(third_dense_layer_two)

new_model = tfk.models.Model(inputs=[vgg_layer_two.input],outputs=[final_dense_layer_two])
for layers in vgg_layer_two.layers:
  layers.trainable=False
new_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)  

In [20]:
new_model.load_weights(r'/content/drive/My Drive/SavedWeights/ImageClassification/IntelImageClassification/model_weights/')

In [21]:
new_model.compile(loss=tfk.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tfk.optimizers.Adam(0.01),metrics=['acc'])
new_model.evaluate(test_data_flow,batch_size=32,return_dict=True)

94/94 [==============================] - 6s 69ms/step - loss: 0.4260 - acc: 0.8363


{'acc': 0.8363333344459534, 'loss': 0.4259969890117645}

In [22]:
vgg_layer_three = tf.keras.applications.VGG16(include_top=False, weights='imagenet',
                                        )
avg_pool_layer_three = tfk.layers.GlobalAveragePooling2D()(vgg_layer_three.output)
first_dense_layer_three = tfk.layers.Dense(256,activation='relu')(avg_pool_layer_three)
second_dense_layer_three = tfk.layers.Dense(128,activation='relu')(first_dense_layer_three)
third_dense_layer_three = tfk.layers.Dense(64,activation='relu')(second_dense_layer_three)
final_dense_layer_three = tfk.layers.Dense(6)(third_dense_layer_three)

new_model_three = tfk.models.Model(inputs=[vgg_layer_three.input],outputs=[final_dense_layer_three])
for layers in vgg_layer_three.layers:
  layers.trainable=False
new_model_three.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)  

In [23]:
new_model_three.load_weights(r'/content/drive/My Drive/SavedWeights/ImageClassification/IntelImageClassification/model_checkpoint/')
new_model_three.compile(loss=tfk.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tfk.optimizers.Adam(0.01),metrics=['acc'])
new_model_three.evaluate(test_data_flow,batch_size=32,return_dict=True)

94/94 [==============================] - 7s 71ms/step - loss: 0.4260 - acc: 0.8363


{'acc': 0.8363333344459534, 'loss': 0.4259969890117645}